<div id='toc'></div>

# Detector efficiency

In [1]:
TFile file("NewEvents.root");

In [ ]:
//genral definition to use Tree
#define PI 3.14159265
TTree* tree =(TTree*) file.Get("events");
Int_t nTracks, detectors[100];
Double_t theta[100],clusterEnergy[100],vetoEnergy[100],phi[100];
Float_t elab[2],ebeam,dircos[2][3],theta_t[2],plab[2];

TLorentzVector Lab_Target;
TLorentzVector Lab_Beam;
TLorentzVector Lab_Initial_State;
TLorentzVector Lab_Proton;
TLorentzVector CM_Proton;


TH1F *h_theta_cm_true =new TH1F("proton theta cm","proton theta cm", 18,0,180);
TH1F *h_true =new TH1F("true neutron energy","true neutron energy", 100,0,1500);
TH1F *h_true_ebeam =new TH1F("true_beamenergy[GeV]","true_beamenergy[GeV]", 100,0,1500);
TH1F *h_true_thetap =new TH1F("true_theta proton","true_theta proton", 18,0,180);
TH1F *h_true_thetan =new TH1F("true_theta neutron","true_theta neutron", 36,0,180);
TH1F *h_true_p =new TH1F("true proton energy","true proton energy", 100,0,1500);
TH2F *h_true_thetap_Eg = new TH2F("theta proton vs Ebeam","theta proton vs Ebeam", 18,0,180, 100,200,1500);
TH2F *h_theta_cm_2 = new TH2F(" proton theta cm  vs true theta","proton tehta cm vs true theta",18,0,180,18,0,180);
TH2F *h_true_thetap_Eg2 = new TH2F("true theta proton cm vs Ebeam"," true theta proton cm vs Ebeam", 18,0,180, 100,200,1500);


tree->SetBranchAddress("nTracks",&nTracks);
tree->SetBranchAddress("detectors",detectors);
tree->SetBranchAddress("theta",theta);
tree->SetBranchAddress("phi",phi);
tree->SetBranchAddress("clusterEnergy",clusterEnergy);
tree->SetBranchAddress("vetoEnergy",vetoEnergy);
tree->SetBranchAddress("elab",elab);
tree->SetBranchAddress("plab",plab);
tree->SetBranchAddress("theta_t",theta_t);
tree->SetBranchAddress("ebeam",&ebeam);
tree->SetBranchAddress("dircos",dircos);


for(int i =0,N= tree->GetEntries(); i< N; ++i){
    tree->GetEntry(i);
    // defining the true refrence values
    h_true_thetap->Fill(theta_t[0]);
    h_true_thetan->Fill(theta_t[1]);
    h_true->Fill(elab[1]*1000-939);
    h_true_ebeam->Fill(ebeam*1000);
    h_true_p->Fill(elab[0]*1000-938);
    h_true_thetap_Eg->Fill(theta_t[0],ebeam*1000);
    
    // boost in cm
    Float_t Md = 1875.62;
    Lab_Beam.SetPxPyPzE(0.0, 0.0, ebeam*1000, ebeam*1000);
    Lab_Target.SetPxPyPzE(0.0, 0.0, 0.0, Md);
 
    Lab_Proton.SetPxPyPzE(dircos[0][0]*plab[0]*1000,dircos[0][1]*plab[0]*1000,dircos[0][2]*plab[0]*1000,elab[0]*1000);
                                             
    //calculate veloctiy
    Lab_Initial_State =  Lab_Target + Lab_Beam;
    Float_t beta;
    beta = Lab_Initial_State.Beta();


    // boost proton with beta
    CM_Proton =  Lab_Proton;
    CM_Proton.Boost(0, 0, -beta);

    // new theta angle in centrre of mass frame
    Float_t theta_p_cm;
    theta_p_cm = CM_Proton.Theta()*180./PI;  
    
    h_theta_cm_true->Fill(theta_p_cm); 
    h_theta_cm_2->Fill(theta_p_cm,theta_t[0]);
    h_true_thetap_Eg2->Fill(theta_p_cm,ebeam*1000);
     
}

In [ ]:
// definition of proton energy, neutron energy and neutron polar angle as a function of
// proton polar angly and beam enrgy
#define PI 3.14159265

Double_t E_n(Double_t Eg,  Double_t zp,Double_t Md=1.875, Double_t Mp=0.938, Double_t Mn=0.939){
     Double_t E=(1.0/2.0)*(-2*std::pow(Eg, 3)*std::pow(zp, 2) + 2*std::pow(Eg, 3) - 2*std::pow(Eg, 2)*Md*std::pow(zp, 2) + 4*std::pow(Eg, 2)*Md + 3*Eg*std::pow(Md, 2) + Eg*std::pow(Mn, 2) - Eg*std::pow(Mp, 2) - Eg*zp*std::sqrt(4*std::pow(Eg, 2)*std::pow(Md, 2) + 4*std::pow(Eg, 2)*std::pow(Mp, 2)*std::pow(zp, 2) - 4*std::pow(Eg, 2)*std::pow(Mp, 2) + 4*Eg*std::pow(Md, 3) - 4*Eg*Md*std::pow(Mn, 2) - 4*Eg*Md*std::pow(Mp, 2) + std::pow(Md, 4) - 2*std::pow(Md, 2)*std::pow(Mn, 2) - 2*std::pow(Md, 2)*std::pow(Mp, 2) + std::pow(Mn, 4) - 2*std::pow(Mn, 2)*std::pow(Mp, 2) + std::pow(Mp, 4)) + std::pow(Md, 3) + Md*std::pow(Mn, 2) - Md*std::pow(Mp, 2))/(-std::pow(Eg, 2)*std::pow(zp, 2) + std::pow(Eg, 2) + 2*Eg*Md + std::pow(Md, 2));
    return E;

} 

In [ ]:
Double_t E_p(Double_t Eg,  Double_t zp,Double_t Md=1.875, Double_t Mp=0.938, Double_t Mn=0.939){
    Double_t Ep=(2*std::pow(Eg, 2)*Md + 3*Eg*std::pow(Md, 2) - Eg*std::pow(Mn, 2) + Eg*std::pow(Mp, 2) + Eg*zp*std::sqrt(4*std::pow(Eg, 2)*std::pow(Md, 2) + 4*std::pow(Eg, 2)*std::pow(Mp, 2)*std::pow(zp, 2) - 4*std::pow(Eg, 2)*std::pow(Mp, 2) + 4*Eg*std::pow(Md, 3) - 4*Eg*Md*std::pow(Mn, 2) - 4*Eg*Md*std::pow(Mp, 2) + std::pow(Md, 4) - 2*std::pow(Md, 2)*std::pow(Mn, 2) - 2*std::pow(Md, 2)*std::pow(Mp, 2) + std::pow(Mn, 4) - 2*std::pow(Mn, 2)*std::pow(Mp, 2) + std::pow(Mp, 4)) + std::pow(Md, 3) - Md*std::pow(Mn, 2) + Md*std::pow(Mp, 2))/(-2*std::pow(Eg, 2)*std::pow(zp, 2) + 2*std::pow(Eg, 2) + 4*Eg*Md + 2*std::pow(Md, 2));
    return Ep;
    }

In [ ]:
Double_t z_n(Double_t Eg,  Double_t zp,Double_t Md=1.875, Double_t Mp=0.938, Double_t Mn=0.939){
    Double_t zn=(Eg - zp*std::sqrt(-std::pow(Mp, 2) + std::pow(Eg + Md - 1.0/2.0*(2*std::pow(Eg, 3)*std::pow(zp, 2) - 2*std::pow(Eg, 3) + 2*std::pow(Eg, 2)*Md*std::pow(zp, 2) - 4*std::pow(Eg, 2)*Md - 3*Eg*std::pow(Md, 2) - Eg*std::pow(Mn, 2) + Eg*std::pow(Mp, 2) + Eg*zp*std::sqrt(4*std::pow(Eg, 2)*std::pow(Md, 2) + 4*std::pow(Eg, 2)*std::pow(Mp, 2)*std::pow(zp, 2) - 4*std::pow(Eg, 2)*std::pow(Mp, 2) + 4*Eg*std::pow(Md, 3) - 4*Eg*Md*std::pow(Mn, 2) - 4*Eg*Md*std::pow(Mp, 2) + std::pow(Md, 4) - 2*std::pow(Md, 2)*std::pow(Mn, 2) - 2*std::pow(Md, 2)*std::pow(Mp, 2) + std::pow(Mn, 4) - 2*std::pow(Mn, 2)*std::pow(Mp, 2) + std::pow(Mp, 4)) - std::pow(Md, 3) - Md*std::pow(Mn, 2) + Md*std::pow(Mp, 2))/(std::pow(Eg, 2)*std::pow(zp, 2) - std::pow(Eg, 2) - 2*Eg*Md - std::pow(Md, 2)), 2)))/std::sqrt(-std::pow(Mn, 2) + (1.0/4.0)*std::pow(2*std::pow(Eg, 3)*std::pow(zp, 2) - 2*std::pow(Eg, 3) + 2*std::pow(Eg, 2)*Md*std::pow(zp, 2) - 4*std::pow(Eg, 2)*Md - 3*Eg*std::pow(Md, 2) - Eg*std::pow(Mn, 2) + Eg*std::pow(Mp, 2) + Eg*zp*std::sqrt(4*std::pow(Eg, 2)*std::pow(Md, 2) + 4*std::pow(Eg, 2)*std::pow(Mp, 2)*std::pow(zp, 2) - 4*std::pow(Eg, 2)*std::pow(Mp, 2) + 4*Eg*std::pow(Md, 3) - 4*Eg*Md*std::pow(Mn, 2) - 4*Eg*Md*std::pow(Mp, 2) + std::pow(Md, 4) - 2*std::pow(Md, 2)*std::pow(Mn, 2) - 2*std::pow(Md, 2)*std::pow(Mp, 2) + std::pow(Mn, 4) - 2*std::pow(Mn, 2)*std::pow(Mp, 2) + std::pow(Mp, 4)) - std::pow(Md, 3) - Md*std::pow(Mn, 2) + Md*std::pow(Mp, 2), 2)/std::pow(std::pow(Eg, 2)*std::pow(zp, 2) - std::pow(Eg, 2) - 2*Eg*Md - std::pow(Md, 2), 2));
    return zn;
}

In [ ]:

TH2F *h_En_Eg = new TH2F("Ebeam_Eneutron","Ebeam_Eneutron", 100,0,2000, 100,0,1500);
TH2F *h_Ep_Eg = new TH2F("Ebeam_Eproton","Ebeam_Eproton", 100,0,2000, 100,0,1500);
TH2F *h_thetap_Eg = new TH2F("theta proton vs Ebeam","theta proton vs Ebeam", 18,0,180, 100,200,1500);
TH1F *h_p =new TH1F("proton","proton", 100,0,1500);
TH1F *h_n =new TH1F("neutron","neutron", 100,0,1500);
TH1F *h_ebeam =new TH1F("beam energy","bem energy", 100,0,1500);
TH1F *h_thetap =new TH1F(" recostructed proton theta"," reconstructed proton theta", 18,0,180);
TH1F *h_thetan =new TH1F(" reconstructed neutron theta"," reconstructed neutron theta", 36,0,180);
TH2F *h_theta_thetap = new TH2F("diffrence of reconstructed and true proton polar angle as a function of reconstructed angle","diffrence of reconstructed and true proton polar angle as a function of reconstructed angle",36,0,180,100,-20,20);
TH1F *h_diff_p =new TH1F("diffrence of reconstructed and true proton energy","diffrence of reconstructed and true proton energy", 100,-80,80);
TH1F *h_diff_p_calc =new TH1F("diffrence of calculated and true proton energy","diffrence of calculated and true proton energy", 100,-30,30);
TH1F *h_diff_n =new TH1F("diffrence of reconstructed and true neutron energy","diffrence of reconstructed and true neutron energy", 100,-30,30);
TH2F *h_true_measured_thetap = new TH2F("true theta va reconstructed","true theta va reconstructed",36,0,180,36,0,180);
TH1F *h_theta_cm =new TH1F("proton theta cm","proton theta cm", 18,0,180);
TH2F *h_theta_cm_1 = new TH2F("cm_theta","cm_ theta",18,0,180,18,0,180);
TH1F *h2 =new TH1F("diffrence true and reconstrcuted proton polar angle","diffrence true and reconstrcuted proton polar angle", 100,-20,20);
TH2F *h_thetap_Eg2 = new TH2F("theta proton vs Ebeam","theta proton vs Ebeam", 18,0,180, 100,200,1500);

TLorentzVector Lab_Target;
TLorentzVector Lab_Beam;
TLorentzVector Lab_Initial_State;
TLorentzVector Lab_Proton;
TLorentzVector CM_Proton;


for(int i =0,N= tree->GetEntries(); i< N; ++i){
    tree->GetEntry(i);
        if(nTracks==2){
        int prot = 0;
        int neut = 1;
            if(vetoEnergy[0]>vetoEnergy[1]){
                prot = 0;
                neut = 1;
            }
            else if(vetoEnergy[0]<vetoEnergy[1]){
                prot = 1;
                neut = 0;
            }
            else if(vetoEnergy[0]==0 && vetoEnergy[1]==0){
                continue; //this to cut the events when both tracks have VetoEnergy ==0
            }
 if (  detectors[prot]==7 or detectors[prot]==3 or detectors[prot]==5 or detectors[prot]==6 or detectors[prot]==40 ){ //       
     if (vetoEnergy[prot]>0.7){ // muss ich detector 5 noch überprüfen, da ich mit der vetoenergy diesen fall ausschließe
                    double zp=cos(theta[prot]* PI / 180.0 );
                   h2->Fill(theta[prot]-theta_t[prot]);
                
                   
                    
                                  if(detectors[neut]==1){
                                     
                                     if(abs(phi[neut])+abs(phi[prot])<=(180+1.5*5.99)){
                                       if( abs(phi[neut])+abs(phi[prot])>=(180-1.5*5.99)){
                                         
                                             auto diff_theta=acos(z_n(ebeam,zp))*(180.0 / PI)-theta[neut];
                                             if (diff_theta<=2*4.95){
                                                 if(diff_theta>=-2*4.95){
                                                     //boost in cm
                                                     Float_t Md = 1875.62;
                                                    Lab_Beam.SetPxPyPzE(0.0, 0.0, ebeam*1000, ebeam*1000);
                                                    Lab_Target.SetPxPyPzE(0.0, 0.0, 0.0, Md);
                                                    double_t relP_p = std::sqrt(std::pow(E_p(ebeam,zp)*1000,2) - std::pow(938,2));
                                                    double_t px = relP_p*cos(phi[prot])*std::sqrt(1-zp*zp);
                                                    double_t py = relP_p*sin(phi[prot])*std::sqrt(1-zp*zp);
                                                    double_t pz = relP_p*zp;
                                                    Lab_Proton.SetPxPyPzE(px, py, pz, E_p(ebeam,zp)*1000);
                                                     //Geschwindigkeit ausrechnen:
                                                    Lab_Initial_State =  Lab_Target + Lab_Beam;
                                                    Float_t beta;
                                                    beta = Lab_Initial_State.Beta();


                                                    // und jetzt das Proton mit -beta boosten
                                                    CM_Proton =  Lab_Proton;
                                                    CM_Proton.Boost(0, 0, -beta);

                                                    // der CM Winkel des Protons sollte dann sein:
                                                    Float_t theta_p_cm;
                                                    theta_p_cm = CM_Proton.Theta()*180./PI;  
                                             h_theta_cm->Fill(theta_p_cm);       
                                             h_p->Fill(E_p(ebeam,zp)*1000-938); 
                                             h_n->Fill(E_n(ebeam,zp)*1000-939);
                                             h_En_Eg->Fill(elab[1]*1000-939,E_n(ebeam,zp)*1000-939);
                                             h_thetap->Fill(theta[prot] );
                                             h_thetan->Fill(acos(z_n(ebeam,zp))*180.0 / PI );
                                             h_ebeam->Fill(ebeam*1000);
                                             h_thetap_Eg->Fill(theta[prot],ebeam*1000);
                                             dircos[0][2] =acos(dircos[0][2])*(180/3.14159265);
                                             h_theta_thetap->Fill(theta[prot],dircos[0][2]-theta[prot]);
                                             h_Ep_Eg->Fill(elab[0]*1000-938,E_p(ebeam,zp)*1000-938);
                                             h_diff_p->Fill(elab[0]*1000-938-clusterEnergy[prot]);
                                             h_diff_p_calc->Fill(elab[0]*1000-E_p(ebeam,zp)*1000);
                                             h_diff_n->Fill(elab[1]*1000-E_n(ebeam,zp)*1000);
                                             h_true_measured_thetap->Fill(theta_t[0],theta[prot]);
                                             h_theta_cm_1->Fill(theta_p_cm,theta_t[0]);
                                             h_thetap_Eg2->Fill(theta_p_cm,ebeam*1000);
                                                 }
                                             }
                                        }  
                                 
                                    }                                 
                                }
                     
                            
                        }
     
            

    
    
                }
            }
        }

    

## polar angle correlation

In [ ]:
// deviation of the reconstructed proton angle from the true angle

TCanvas c2;
//gStyle->SetOptStat(0);
h2->SetTitle("difference proton theta angle ");
h2->GetYaxis()->SetTitle(""); 
h2->GetXaxis()->SetTitle(" difference #theta_{p} [#circ]");
h2->Fit("gaus");
h2->Draw();
c2.SetGrid();
c2.Draw();



In [ ]:
   int bin1 = h2->FindFirstBinAbove( h2->GetMaximum()/2);
   int bin2 = h2->FindLastBinAbove( h2->GetMaximum()/2);
   double fwhm = h2->GetBinCenter(bin2) - h2->GetBinCenter(bin1);
cout<<"the fwhm is "<<fwhm<<endl<<fwhm/2.3548;

In [ ]:
TCanvas c2;
h_true_measured_thetap->SetYTitle(" theta_{P}, [#circ]");
h_true_measured_thetap->SetXTitle(" theta_{true P}, [#circ]");
h_true_measured_thetap->SetTitle("");
h_true_measured_thetap->Draw("colz");
//gStyle->SetOptStat(0); 

c2.Draw();


In [ ]:
TCanvas c2;
h_theta_thetap->GetXaxis()->SetTitle("measured theta [#circ]");
h_theta_thetap->GetYaxis()->SetTitle("diffence measured and true theta [#circ]"); 
h_theta_thetap->SetTitle("diffrence-theta vs measured theta");
h_theta_thetap->Draw("colz");
c2.SetGrid();
 c2.Draw();


## energy correlation

### true energy vs measured for proton

In [ ]:
TCanvas c2;

h_diff_p->SetTitle("");
h_diff_p->GetYaxis()->SetTitle(""); 
h_diff_p->GetXaxis()->SetTitle(" difference [MeV]"); 
//h_diff_p->GetYaxis()->SetRangeUser(0,70000);
//h_diff_p->GetXaxis()->SetRangeUser(-60,60);
h_diff_p->Fit("gaus");
h_diff_p->Draw();

c2.SetGrid();
 c2.Draw();


### true proton energy vs calculated proton

In [ ]:
TF1 *fp=new TF1("fp","gaus",-20,20);
TCanvas c2;

h_diff_p_calc->SetTitle("");
h_diff_p_calc->GetYaxis()->SetTitle(""); 
h_diff_p_calc->GetXaxis()->SetTitle(" difference [MeV]"); 
//h_diff_p->GetYaxis()->SetRangeUser(0,70000);
//h_diff_p->GetXaxis()->SetRangeUser(-60,60);
h_diff_p_calc->Fit("fp","R");
h_diff_p_calc->Draw();

c2.SetGrid();
 c2.Draw();


### true neutron energy vs calculated neutron energy

In [ ]:
TF1 *fp=new TF1("fp","gaus",-20,20);
TCanvas c2;

h_diff_n->SetTitle("");
h_diff_n->GetYaxis()->SetTitle(""); 
h_diff_n->GetXaxis()->SetTitle(" difference [MeV]"); 
//h_diff_n->GetYaxis()->SetRangeUser(0,70000);
//h_diff_n->GetXaxis()->SetRangeUser(-60,60);
h_diff_n->Fit("fp","R");
h_diff_n->Draw();

c2.SetGrid();
 c2.Draw();

c2.SaveAs("neutE_trueE.jpg")

## total detetcor efficiency 

### efficiency as a function of neutron energy

In [ ]:


h_n->SetTitle("neutron energy ");
h_n->SetTitle("");
h_n->SetLineColor(kGreen);

h_n->GetXaxis()->SetTitle(" neutron kinetic energy [MeV]"); 
//h_n->GetYaxis()->SetTitle(" "); 
TCanvas* c= new TCanvas();
h_n->GetYaxis()->SetRangeUser(0,60000);
h_n->Draw();
h_true->Draw("Same");
//gStyle->SetOptStat(0);
c->Draw();
   auto legend = new TLegend(0.5,0.7,0.88,0.9);
   legend->SetHeader("","C"); // option "C" allows to center the header
   legend->AddEntry(h_n," reconstructed events","l");
   legend->AddEntry(h_true,"true events","l");

   legend->Draw();


The histogramms show the amount of events detected for given neutron energies. The number of events drops for higher neutron energyies. Espacially measured events are nearly zero for neutron energies higher as 1GeV, as the energy of the corresponding proton is not high enough to be detected(?).



In [ ]:
h_n->Sumw2();
auto div_E_tot = new TH1F(*h_n);
 div_E_tot->Divide(h_true);
TCanvas c3;
div_E_tot->GetXaxis()->SetTitle(" neutron kinetic energy [MeV]"); 
div_E_tot->GetYaxis()->SetTitle(" Detection efficiency"); 
div_E_tot->SetLineColor(kRed);
div_E_tot->GetYaxis()->SetRangeUser(0,0.3);
div_E_tot->SetTitle("total detector acceptance vs. neutron energy");
div_E_tot->SetTitle("");
div_E_tot->Draw();
//gStyle->SetOptStat(0);
c3.SetGrid();
 c3.Draw();


### efficiency as a function of proton energy

In [ ]:

h_p->SetLineColor(kGreen);
h_p->GetXaxis()->SetTitle(" proton kinetic energy [MeV]"); 
h_p->GetYaxis()->SetTitle(" ");
h_p->SetTitle("proton energy");
h_p->SetTitle("");
TCanvas* c= new TCanvas();
h_p->GetYaxis()->SetRangeUser(0,60000);
h_p->Draw();
h_true_p->Draw("Same");
//gStyle->SetOptStat(0);
c->Draw();
   auto legend = new TLegend(0.5,0.7,0.88,0.9);
   legend->SetHeader("","C"); // option "C" allows to center the header
   legend->AddEntry(h_p," reconstructed events","l");
   legend->AddEntry(h_true_ebeam," true events","l");

   legend->Draw();


In [ ]:
h_p->Sumw2();
auto div_Ep = new TH1F(*h_p);
 div_Ep->Divide(h_true_p);
TCanvas c9;
div_Ep->GetXaxis()->SetTitle(" proton kinetic energy [MeV]"); 
div_Ep->GetYaxis()->SetTitle(" Detection efficiency"); 

div_Ep->SetLineColor(kRed);
div_Ep->GetYaxis()->SetRangeUser(0,0.3);
div_Ep->SetTitle("total detector acceptance vs. proton energy");
div_Ep->SetTitle("");
div_Ep->Draw();
c9.SetGrid();
 c9.Draw();


### efficiency as a function of the polar angle for the proton

In [ ]:

h_thetap->SetLineColor(kGreen);
h_thetap->GetXaxis()->SetTitle("theta angle[#circ]"); 
h_thetap->GetYaxis()->SetTitle(" ");
h_true_thetap->SetTitle("proton theta angle");
h_true_thetap->SetTitle("");
TCanvas* c= new TCanvas();
h_true_thetap->GetXaxis()->SetTitle(" #theta angle[#circ]"); 
h_true_thetap->Draw();
h_thetap->Draw("Same");



c->Draw();
   auto legend = new TLegend(0.5,0.7,0.88,0.9);
  // legend->SetHeader("","C"); // option "C" allows to center the header
   legend->AddEntry(h_thetap," reconstructed events","l");
   legend->AddEntry(h_true_thetap,"true events","l");

   legend->Draw();


In [ ]:
h_thetap->Sumw2();
auto div_thetap = new TH1F(*h_thetap);
 div_thetap->Divide(h_true_thetap);
TCanvas c5;
div_thetap->GetXaxis()->SetTitle(" #theta angle[#circ]"); 
div_thetap->GetYaxis()->SetTitle(" Detection efficiency"); 
div_thetap->SetLineColor(kRed);
div_thetap->GetYaxis()->SetRangeUser(0,0.25);
div_thetap->SetTitle("total detector acceptance vs. theta angle proton lab");
div_thetap->SetTitle("");
div_thetap->Draw();
c5.SetGrid();
 c5.Draw();


### efficiency as a function of the proton polar angle in CM

In [ ]:

h_theta_cm->SetLineColor(kGreen);
h_theta_cm->GetXaxis()->SetTitle("theta angle[#circ]"); 
h_theta_cm->GetYaxis()->SetTitle(" ");
h_theta_cm_true->SetTitle(" #theta angle in cm");
h_theta_cm_true->SetTitle("");
TCanvas* c= new TCanvas();
h_theta_cm_true->GetXaxis()->SetTitle(" theta angle[#circ]"); 
h_theta_cm_true->Draw();
h_theta_cm->Draw("Same");



c->Draw();
   auto legend = new TLegend(0.5,0.7,0.88,0.9);
  // legend->SetHeader("","C"); // option "C" allows to center the header
   legend->AddEntry(h_theta_cm," reconstructed events","l");
   legend->AddEntry(h_true_thetap,"true events","l");

   legend->Draw();


In [ ]:
h_theta_cm->Sumw2();
auto div_thetap_cm = new TH1F(*h_theta_cm);
 div_thetap_cm->Divide(h_theta_cm_true);
TCanvas c5;
div_thetap_cm->GetXaxis()->SetTitle(" #theta angle[#circ]"); 
div_thetap_cm->GetYaxis()->SetTitle(" Detection efficiency"); 
div_thetap_cm->SetLineColor(kRed);
div_thetap_cm->GetYaxis()->SetRangeUser(0,0.3);

//div_thetap_cm->GetXaxis()->SetRangeUser(30,55);
div_thetap_cm->SetTitle("total detector acceptance vs. theta angle proton");
div_thetap_cm->SetTitle("");
div_thetap_cm->Draw();
c5.SetGrid();
 c5.Draw();


### efficiency as a function of the neutron polar angle

In [ ]:

h_thetan->SetLineColor(kGreen);
h_thetan->GetXaxis()->SetTitle("theta angle[#circ]"); 
h_thetan->GetYaxis()->SetTitle(" ");
h_thetan->SetTitle("neutron polar angle");

TCanvas* c= new TCanvas();

h_true_thetan->GetYaxis()->SetRangeUser(0,57000);
h_true_thetan->Draw();
h_thetan->Draw("Same");



c->Draw();
   auto legend = new TLegend(0.5,0.7,0.88,0.9);
  // legend->SetHeader("","C"); // option "C" allows to center the header
   legend->AddEntry(h_thetan," reconstructed events","l");
   legend->AddEntry(h_true_thetan,"true events","l");

   legend->Draw();

In [ ]:
h_thetan->Sumw2();
auto div_thetan = new TH1F(*h_thetan);
 div_thetan->Divide(h_true_thetan);
TCanvas c5;
div_thetan->GetXaxis()->SetTitle(" theta angle[#circ]"); 
div_thetan->GetYaxis()->SetTitle(" Detection efficiency"); 
div_thetan->SetLineColor(kRed);
//div_thetan->GetYaxis()->SetRangeUser(0,0.3);
div_thetan->SetTitle("efficiency as a function of the polar angle");
div_thetan->Draw();
c5.SetGrid();
 c5.Draw();

### efficiency as a function of beam energy

In [ ]:


h_ebeam->SetLineColor(kGreen);
h_ebeam->GetXaxis()->SetTitle("beam energy [GeV]"); 
h_ebeam->GetYaxis()->SetTitle(" ");
h_ebeam->SetTitle("measured events vs beam energy");
TCanvas* c= new TCanvas();

h_ebeam->GetYaxis()->SetRangeUser(0,45000);
h_ebeam->Draw();
h_true_ebeam->Draw("Same");

c->Draw();
   auto legend = new TLegend(0.5,0.7,0.88,0.9);
   legend->SetHeader("","C"); // option "C" allows to center the header
   legend->AddEntry(h_ebeam," reconstructed events","l");
   legend->AddEntry(h_true_ebeam,"true events","l");

   legend->Draw();

In [ ]:
h_ebeam->Sumw2();

auto div_ebeam = new TH1F(*h_ebeam);
 div_ebeam->Divide(h_true_ebeam);
TCanvas c4;
div_ebeam->GetXaxis()->SetTitle(" beam energy[GeV]"); 
div_ebeam->GetYaxis()->SetTitle(" Detection efficiency"); 
div_ebeam->SetLineColor(kRed);
div_ebeam->GetYaxis()->SetRangeUser(0,0.3);
div_ebeam->SetTitle("efficiency as a function od beam energy");
c4.SetGrid();
div_ebeam->Draw("E");
c4.Draw();

### 2-dim Histogramm: efficiency as a function of polar angle and beam energy

In [ ]:


auto  acc_thetap_Eg = new TH2F(*h_thetap_Eg);
 acc_thetap_Eg->Divide(h_true_thetap_Eg);
TCanvas c4;
//gStyle->SetOptStat(0);
acc_thetap_Eg->GetXaxis()->SetTitle(" proton polar angle #theta[#circ]"); 
acc_thetap_Eg->GetYaxis()->SetTitle(" beam energy [MeV]"); 
//acc_thetap_Eg->GetYaxis()->SetRangeUser(0.57,0.600);

acc_thetap_Eg->SetTitle("total detector acceptance for proton theta vs. beam energy");
acc_thetap_Eg->SetTitle("");
acc_thetap_Eg->Draw("CONTZ");
c4.SetGrid();
c4.Draw();
  

### correlation of the proton polar angle in lab and cm 

In [ ]:
auto  acc_theta = new TH2F(*h_theta_cm_1);
 acc_theta->Divide(h_theta_cm_2);
TCanvas c4;
//gStyle->SetOptStat(0);
acc_theta->GetXaxis()->SetTitle(" cm theta angle[#circ]"); 
acc_theta->GetYaxis()->SetTitle(" lab theta angle [#circ]"); 
//acc_thetap_Eg->GetYaxis()->SetRangeUser(0.57,0.600);

acc_theta->SetTitle("total detector acceptance for cm theta vs. lab theta");
acc_theta->SetTitle("tehta lab vs. theta cm");
acc_theta->Draw("CONTZ");
c4.SetGrid();
c4.Draw();
//c4.SaveAs("theta_lab_cm.jpg");  

###  2-dim Histogramm: efficiency as a function of proton polar angle cm and beam energy

In [ ]:


auto  acc_thetap_Eg2 = new TH2F(*h_thetap_Eg2);
 acc_thetap_Eg2->Divide(h_true_thetap_Eg2);
TCanvas c4;
//gStyle->SetOptStat(0);
acc_thetap_Eg2->GetXaxis()->SetTitle(" proton polar angle #theta_{cm}[#circ]"); 
acc_thetap_Eg2->GetYaxis()->SetTitle(" beam energy [MeV]"); 
//acc_thetap_Eg->GetYaxis()->SetRangeUser(0.57,0.600);

acc_thetap_Eg2->SetTitle("total detector acceptance for proton theta vs. beam energy");
acc_thetap_Eg2->SetTitle("");
acc_thetap_Eg2->Draw("CONTZ");
c4.SetGrid();
c4.Draw();
 